In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import gc
gc.collect()

# SetUp

In [ ]:
!pip install -q -U -r requirements.txt
# !pip install -q -U keras-cv

# Import Required Packages

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from collections import defaultdict
import numpy as np
from PIL import Image
import shutil
from tqdm import tqdm
import random

import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras import layers, models, applications, optimizers
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, accuracy_score, precision_score, recall_score, f1_score


# Constants

In [ ]:
DIR_PATH = '/content/drive/MyDrive/Colab Notebooks/'
TEST_DATA_DIR = DIR_PATH + "test_dataset/"
TRAIN_DATA_DIR = DIR_PATH + "train_dataset/"
AUGMENTED_DIR = DIR_PATH + "augmented_train_dataset"
AUTOTUNE = tf.data.AUTOTUNE

IMG_SIZE = (75, 75)
INPUT_SHAPE = IMG_SIZE + (3,)
EPOCHS = 50
BATCH_SIZE = 32
BUFFER_SIZE = 100

# Constants


# Load Augmented and Test Data

In [ ]:
# Load datasets
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    AUGMENTED_DIR,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='binary'
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    AUGMENTED_DIR,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='binary'
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TEST_DATA_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False  # Keep order for metrics/report
)

In [ ]:
num_classes = len(train_ds.class_names)
num_classes

# Preprocessing Dataset
- convert image to grayscale
- Rescale: rescale pixel values
- Resize:  images to a consistent shape (with tf.keras.layers.Resizing)

In [ ]:
# Define normalization transformation
def normalization(image, label):
    # Apply Rescaling (normalize)
    rescale = tf.keras.layers.Rescaling(1./255)
    image = rescale(image)
    return image, label

In [ ]:
# Apply transformation
train_ds = train_ds.map(normalization, num_parallel_calls=AUTOTUNE).cache().prefetch(buffer_size=AUTOTUNE)
val_ds   = val_ds.map(normalization, num_parallel_calls=AUTOTUNE).cache().prefetch(buffer_size=AUTOTUNE)
test_ds  = test_ds.map(normalization, num_parallel_calls=AUTOTUNE).prefetch(buffer_size=AUTOTUNE)

In [ ]:
# Verify that the pixels are in the [0, 1] range:
for images, labels in test_ds.take(1):
    print("Min pixel value:", tf.reduce_min(images).numpy())
    print("Max pixel value:", tf.reduce_max(images).numpy())

In [ ]:
print("Image shape:", images.shape)

# Hyperparameter Tuning
## set up a hypermodel

Hyperparamerters:

- Number of filters in each Conv2D layer
- Dropout rate
- Dense layer units
- Learning rate
- Optimizer

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Define model checkpoint callback
def create_checkpoint_callback(model_name):
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path,
        monitor='val_loss',   # or 'val_accuracy' depending on your goal
        save_best_only=True,
        save_weights_only=False,
        mode='min',           # or 'max' if you're monitoring accuracy
        verbose=1
    )
    return checkpoint_cb

## 1. Define Baseline Model: CNN

In [ ]:
def build_cnn_model_tuning(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))) # Input Layer (Grayscale to 1 channel)

    # Conv Layers
    for i in range(3):
        model.add(tf.keras.layers.Conv2D(
            filters=hp.Int(f'conv_{i}_filters', min_value=32, max_value=256, step=32),
            kernel_size=hp.Choice('kernel_size', [3, 5]),
            activation='relu',
            padding='same'))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.MaxPooling2D())

    # Dropout
    model.add(tf.keras.layers.Dropout(hp.Float('dropout_rate', 0.2, 0.5, step=0.1)))

    # Flatten or GAP
    model.add(tf.keras.layers.GlobalAveragePooling2D())

    # Dense Layer
    model.add(tf.keras.layers.Dense(units=hp.Int('dense_units',  min_value=32, max_value=256, step=64),
                                    activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001),
))
    model.add(tf.keras.layers.Dropout(hp.Float('dense_dropout', 0.2, 0.5, step=0.1)))

    # Output Layer
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    # Compile
    model.compile(
        optimizer=hp.Choice('optimizer', ['adam']),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model


## Hyperband tuner

In [ ]:
def run_tuner(model_name, build_model):
  hyperband_tuner = kt.Hyperband(
      build_model,
      objective='val_accuracy',
      max_epochs=EPOCHS,
      factor=3,
      executions_per_trial=1,
      hyperband_iterations=1,
      directory='tuner_dir',
      project_name=model_name,
      overwrite=True
      )
  return hyperband_tuner

## 2. Define Transfer Learning Models

In [ ]:
def build_tl_model(base_model):
    def build_transfer_model(hp):

        base_model.trainable = hp.Boolean('fine_tune', default=False)

        model = tf.keras.models.Sequential([
            base_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(units=hp.Int('dense_units',  min_value=32, max_value=256, step=64),
                                    activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001)),
            tf.keras.layers.Dropout(hp.Float('dense_dropout', 0.2, 0.5, step=0.1)),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])

        # Compile
        model.compile(
            optimizer=hp.Choice('optimizer', ['adam']),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )

        return model
    return build_transfer_model


## Define Model Evaluation

In [ ]:
def model_evaluation(best_model, results, model_name):
    test_preds = best_model.predict(test_ds)

    # Handle binary vs. multi-class prediction
    if test_preds.shape[-1] == 1:
        # Binary classification
        test_preds = test_preds.flatten()
        test_preds_label = (test_preds > 0.5).astype(int)
    else:
        # Multi-class classification
        test_preds_label = np.argmax(test_preds, axis=1)

    # Get true labels
    test_labels = np.concatenate([y.numpy() for x, y in test_ds])

    # Ensure matching lengths
    assert test_labels.shape[0] == test_preds_label.shape[0], f"Mismatched: {test_labels.shape} vs {test_preds_label.shape}"

    # Compute metrics
    acc = accuracy_score(test_labels, test_preds_label)
    prec = precision_score(test_labels, test_preds_label, average='binary' if len(np.unique(test_labels)) == 2 else 'macro')
    rec = recall_score(test_labels, test_preds_label, average='binary' if len(np.unique(test_labels)) == 2 else 'macro')
    f1 = f1_score(test_labels, test_preds_label, average='binary' if len(np.unique(test_labels)) == 2 else 'macro')
    auc = roc_auc_score(test_labels, test_preds if test_preds.ndim == 1 else test_preds[:,1])

    # Print classification report
    print(f"Model: {model_name}")
    print(classification_report(test_labels, test_preds_label))

    # Confusion matrix
    plt.figure(figsize=(6, 5))
    sns.heatmap(confusion_matrix(test_labels, test_preds_label), annot=True, fmt='d', cmap='Blues')
    plt.title(f"Confusion Matrix - {model_name}")
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

    # ROC Curve
    fpr, tpr, _ = roc_curve(test_labels, test_preds if test_preds.ndim == 1 else test_preds[:,1])
    plt.figure()
    plt.plot(fpr, tpr, label=f'{model_name} (AUC = {auc:.2f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc='lower right')
    plt.grid(True)
    plt.show()

    # Save results
    results[model_name] = {
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1,
        "auc": auc
    }

    return results

# Model Training and Evaluation

In [ ]:
results = {}

## 1. Baseline Model: AlexNet

In [ ]:
model_name = "CNN/AlexNet"

In [ ]:
hyperband_tuner = run_tuner(model_name, build_cnn_model_tuning)

checkpoint_path = f"{DIR_PATH}saved_models/best_checkpoint_{model_name}.h5"
checkpoint_cb = create_checkpoint_callback(model_name)

# Run Search
history = hyperband_tuner.search(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=[early_stop, checkpoint_cb])

# Train Best Model
cnn_best_hps = hyperband_tuner.get_best_hyperparameters(num_trials=1)[0]

cnn_best_model = hyperband_tuner.hypermodel.build(cnn_best_hps)
cnn_best_model.summary()

## Save Best Model
cnn_best_model.save(f"{DIR_PATH}saved_models/best_model_{model_name}.h5")

In [ ]:
# Retrieve Best Hyperparameters
print(f"Best Hyperparameters for {model_name.upper()}:")
best_hps = cnn_best_hps.values

In [ ]:
#### Plot Accuracy and Loss Curves
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(epochs_range, acc, label='Train Acc')
plt.plot(epochs_range, val_acc, label='Val Acc')
plt.title(f'{model_name} Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(epochs_range, loss, label='Train Loss')
plt.plot(epochs_range, val_loss, label='Val Loss')
plt.title(f'{model_name} Loss')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Model Evaluation
results = model_evaluation(cnn_best_model,results,model_name)

## 2. VGG16

In [ ]:
model_name = "VGG16"

In [ ]:
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

In [ ]:

# Get a Keras Tuner-compatible model builder
model_builder = build_tl_model(base_model)
hyperband_tuner = run_tuner(model_name, model_builder)

# Run Search
history = hyperband_tuner.search(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=[early_stop, checkpoint_cb])

# Train Best Model
vgg16_best_hps = hyperband_tuner.get_best_hyperparameters(num_trials=1)[0]

vgg16_best_model = hyperband_tuner.hypermodel.build(vgg16_best_hps)
vgg16_best_model.summary()

## Save Best Model
vgg16_best_model.save(f"saved_models/best_model_{model_name}.h5")

In [ ]:
# Retrieve Best Hyperparameters
print(f"Best Hyperparameters for {model_name.upper()}:")
print(vgg16_best_hps.values)

In [ ]:
#### Plot Accuracy and Loss Curves
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(epochs_range, acc, label='Train Acc')
plt.plot(epochs_range, val_acc, label='Val Acc')
plt.title(f'{model_name} Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(epochs_range, loss, label='Train Loss')
plt.plot(epochs_range, val_loss, label='Val Loss')
plt.title(f'{model_name} Loss')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Model Evaluation
results = model_evaluation(vgg16_best_model, results, model_name)

## 3. ResNet50

In [ ]:
# Define constants
model_name = "ResNet50"
base_model = applications.ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

In [ ]:
# Get a Keras Tuner-compatible model builder
model_builder = build_tl_model(base_model)
hyperband_tuner = run_tuner(model_name, model_builder)

# Run Search
history = hyperband_tuner.search(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=[early_stop, checkpoint_cb])

# Train Best Model
restnet50_best_hps = hyperband_tuner.get_best_hyperparameters(num_trials=1)[0]

restnet50_best_model = hyperband_tuner.hypermodel.build(restnet50_best_hps)
restnet50_best_model.summary()

## Save Best Model
restnet50_best_model.save(f"saved_models/best_model_{model_name}.h5")

In [ ]:
# Retrieve Best Hyperparameters
print(f"Best Hyperparameters for {model_name.upper()}:")
print(restnet50_best_hps.values)

In [ ]:
#### Plot Accuracy and Loss Curves
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(epochs_range, acc, label='Train Acc')
plt.plot(epochs_range, val_acc, label='Val Acc')
plt.title(f'{model_name} Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(epochs_range, loss, label='Train Loss')
plt.plot(epochs_range, val_loss, label='Val Loss')
plt.title(f'{model_name} Loss')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Model Evaluation
results = model_evaluation(restnet50_best_model, results, model_name)

## 4. InceptionV3

In [ ]:
model_name = "InceptionV3"
base_model = applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

In [ ]:
# Get a Keras Tuner-compatible model builder
model_builder = build_tl_model(base_model)
hyperband_tuner = run_tuner(model_name, model_builder)

# Run Search
history = hyperband_tuner.search(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=[early_stop, checkpoint_cb])

# Train Best Model
InceptionV3_best_hps = hyperband_tuner.get_best_hyperparameters(num_trials=1)[0]

InceptionV3_best_model = hyperband_tuner.hypermodel.build(InceptionV3_best_hps)
InceptionV3_best_model.summary()

## Save Best Model
InceptionV3_best_model.save(f"saved_models/best_model_{model_name}.h5")

In [ ]:
# Retrieve Best Hyperparameters
print(f"Best Hyperparameters for {model_name.upper()}:")
print(InceptionV3_best_hps.values)

In [ ]:
#### Plot Accuracy and Loss Curves
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(epochs_range, acc, label='Train Acc')
plt.plot(epochs_range, val_acc, label='Val Acc')
plt.title(f'{model_name} Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(epochs_range, loss, label='Train Loss')
plt.plot(epochs_range, val_loss, label='Val Loss')
plt.title(f'{model_name} Loss')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Model Evaluation
results = model_evaluation(InceptionV3_best_model, results, model_name)

## 5. EfficientNetB0

In [ ]:
model_name = "EfficientNetB0"
base_model = applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

In [ ]:
# Get a Keras Tuner-compatible model builder
model_builder = build_tl_model(base_model)
hyperband_tuner = run_tuner(model_name, model_builder)

# Run Search
history = hyperband_tuner.search(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=[early_stop, checkpoint_cb])

# Train Best Model
EfficientNetB0_best_hps = hyperband_tuner.get_best_hyperparameters(num_trials=1)[0]

EfficientNetB0_best_model = hyperband_tuner.hypermodel.build(EfficientNetB0_best_hps)
EfficientNetB0_best_model.summary()

## Save Best Model
EfficientNetB0_best_model.save(f"saved_models/best_model_{model_name}.h5")


In [ ]:
# Retrieve Best Hyperparameters
print(f"Best Hyperparameters for {model_name.upper()}:")
print(EfficientNetB0_best_hps.values)

In [ ]:
#### Plot Accuracy and Loss Curves
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(epochs_range, acc, label='Train Acc')
plt.plot(epochs_range, val_acc, label='Val Acc')
plt.title(f'{model_name} Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(epochs_range, loss, label='Train Loss')
plt.plot(epochs_range, val_loss, label='Val Loss')
plt.title(f'{model_name} Loss')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Model Evaluation
results = model_evaluation(EfficientNetB0_best_model, results, model_name)

## 6. Xception

In [ ]:
model_name = "Xception"
base_model = applications.Xception(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

In [ ]:
# Get a Keras Tuner-compatible model builder
model_builder = build_tl_model(base_model)
hyperband_tuner = run_tuner(model_name, model_builder)

# Run Search
history = hyperband_tuner.search(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=[early_stop, checkpoint_cb])

# Train Best Model
xception_best_hps = hyperband_tuner.get_best_hyperparameters(num_trials=1)[0]

xception_best_model = hyperband_tuner.hypermodel.build(xception_best_hps)
xception_best_model.summary()

## Save Best Model
xception_best_model.save(f"saved_models/best_model_{model_name}.h5")


In [ ]:
# Retrieve Best Hyperparameters
print(f"Best Hyperparameters for {model_name.upper()}:")
print(xception_best_hps.values)

In [ ]:
#### Plot Accuracy and Loss Curves
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(epochs_range, acc, label='Train Acc')
plt.plot(epochs_range, val_acc, label='Val Acc')
plt.title(f'{model_name} Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(epochs_range, loss, label='Train Loss')
plt.plot(epochs_range, val_loss, label='Val Loss')
plt.title(f'{model_name} Loss')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Model Evaluation
results = model_evaluation(xception_model, results, model_name)

## 7. DenseNet121

In [ ]:
model_name = "DenseNet121"
base_model = applications.DenseNet121(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

In [ ]:
# Get a Keras Tuner-compatible model builder
model_builder = build_tl_model(base_model)
hyperband_tuner = run_tuner(model_name, model_builder)

# Run Search
history = hyperband_tuner.search(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=[early_stop, checkpoint_cb])

# Train Best Model
DenseNet121_best_hps = hyperband_tuner.get_best_hyperparameters(num_trials=1)[0]

DenseNet121_best_model = hyperband_tuner.hypermodel.build(DenseNet121_best_hps)
DenseNet121_best_model.summary()

## Save Best Model
DenseNet121_best_model.save(f"saved_models/best_model_{model_name}.h5")

In [ ]:
# Retrieve Best Hyperparameters
print(f"Best Hyperparameters for {model_name.upper()}:")
print(DenseNet121_best_hps.values)

In [ ]:
#### Plot Accuracy and Loss Curves
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(epochs_range, acc, label='Train Acc')
plt.plot(epochs_range, val_acc, label='Val Acc')
plt.title(f'{model_name} Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(epochs_range, loss, label='Train Loss')
plt.plot(epochs_range, val_loss, label='Val Loss')
plt.title(f'{model_name} Loss')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Model Evaluation
results = model_evaluation(DenseNet121_model, results, model_name)

## 8. MobileNetV2

In [ ]:
model_name = "MobileNetV2"
base_model = applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

In [ ]:
# Get a Keras Tuner-compatible model builder
model_builder = build_tl_model(base_model)
hyperband_tuner = run_tuner(model_name, model_builder)

# Run Search
history = hyperband_tuner.search(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=[early_stop, checkpoint_cb])

# Train Best Model
MobileNetV2_best_hps = hyperband_tuner.get_best_hyperparameters(num_trials=1)[0]

MobileNetV2_best_model = hyperband_tuner.hypermodel.build(MobileNetV2_best_hps)
MobileNetV2_best_model.summary()

## Save Best Model
MobileNetV2_best_model.save(f"saved_models/best_model_{model_name}.h5")

In [ ]:
# Retrieve Best Hyperparameters
print(f"Best Hyperparameters for {model_name.upper()}:")
print(MobileNetV2_best_hps.values)

In [ ]:
#### Plot Accuracy and Loss Curves
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(epochs_range, acc, label='Train Acc')
plt.plot(epochs_range, val_acc, label='Val Acc')
plt.title(f'{model_name} Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(epochs_range, loss, label='Train Loss')
plt.plot(epochs_range, val_loss, label='Val Loss')
plt.title(f'{model_name} Loss')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Model Evaluation
results = model_evaluation(MobileNetV2_model, results, model_name)

## 9. NASNetMobile

In [ ]:
model_name = "NASNetMobile"
base_model = applications.NASNetMobile(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

In [ ]:
# Get a Keras Tuner-compatible model builder
model_builder = build_tl_model(base_model)
hyperband_tuner = run_tuner(model_name, model_builder)

# Run Search
history = hyperband_tuner.search(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=[early_stop, checkpoint_cb])

# Train Best Model
NASNetMobile_best_hps = hyperband_tuner.get_best_hyperparameters(num_trials=1)[0]

NASNetMobile_best_model = hyperband_tuner.hypermodel.build(NASNetMobile_best_hps)
NASNetMobile_best_model.summary()

## Save Best Model
NASNetMobile_best_model.save(f"saved_models/best_model_{model_name}.h5")

In [ ]:
# Retrieve Best Hyperparameters
print(f"Best Hyperparameters for {model_name.upper()}:")
print(NASNetMobile_best_hps.values)

In [ ]:
#### Plot Accuracy and Loss Curves
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(epochs_range, acc, label='Train Acc')
plt.plot(epochs_range, val_acc, label='Val Acc')
plt.title(f'{model_name} Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(epochs_range, loss, label='Train Loss')
plt.plot(epochs_range, val_loss, label='Val Loss')
plt.title(f'{model_name} Loss')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Model Evaluation
results = model_evaluation(NASNetMobile_model, results, model_name)

In [ ]:
import pandas as pd

# Assuming your results dictionary is named `results`
df_results = pd.DataFrame(results).T.reset_index()
df_results.columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC']

df_results = df_results.round(4)

print(df_results)